In [1]:
import pandas as pd
import sys
import os
sys.path.append(os.path.dirname(sys.path[0]))
from ast import literal_eval
from src.datasets.ie_hyperion_dataset import find_word_bounds

df = pd.read_csv('../data/processed/pipeline/test/ie_hyperion.csv', converters={'Stralci': literal_eval, 'Repertori': literal_eval})
df = df.head(100)
df['Bounds'] = df.apply(lambda x: find_word_bounds(x['Stralci'], x['Testo']), axis=1).values.tolist()

## Compute predictions with NLTK segmenter

In [2]:
from src.models.nltk_segmenter import NLTKSegmenter


nltk_seg = NLTKSegmenter()
df['Stralci_predetti'] = df['Testo'].map(nltk_seg.predict).values.tolist()
df['Bounds_predetti'] = df.apply(lambda x: find_word_bounds(x['Stralci_predetti'], x['Testo']), axis=1).values.tolist()

[nltk_data] Downloading package punkt to /Users/michele/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
from src.models.bert_segmenter import BertSegmenter

bert_seg = BertSegmenter()
df['Stralci_predetti'] = df['Testo'].map(bert_seg.predict).values.tolist()
df['Bounds_predetti'] = df.apply(lambda x: find_word_bounds(x['Stralci_predetti'], x['Testo']), axis=1).values.tolist()

## Repertories classificaiton

In [3]:
from src.models.bert_rep import BertRep

bert_rep = BertRep()
df['Repertori_predetti'] = df['Stralci_predetti'].map(bert_rep.predict).values.tolist()

## Metrics

In [5]:
def intersection(A, B):
    if A == B:
        return 1
    start = max(A[0], B[0])
    end = min(A[1], B[1])
    if(start > end):
        return 0
    return end - start + 1

def C(pred_bound:tuple, gt_bound:tuple, pred_rep:str, gt_rep:str, norm_factor:int) -> float:
    if pred_rep != gt_rep:
        return 0
    return intersection(pred_bound, gt_bound) / norm_factor

def precision(pred_bounds:list, gt_bounds:list, pred_reps:list, gt_reps:list) -> float:
    curr_sum = 0
    for i in range(len(pred_bounds)):
        for j in range(len(gt_bounds)):
            curr_sum += C(pred_bounds[i], gt_bounds[j], pred_reps[i], gt_reps[j], pred_bounds[i][1] - pred_bounds[i][0] + 1)
    return curr_sum / len(pred_bounds)

def recall(pred_bounds:list, gt_bounds:list, pred_reps:list, gt_reps:list) -> float:
    curr_sum = 0
    for i in range(len(pred_bounds)):
        for j in range(len(gt_bounds)):
            curr_sum += C(pred_bounds[i], gt_bounds[j], pred_reps[i], gt_reps[j], gt_bounds[j][1] - gt_bounds[j][0] + 1)
    return curr_sum / len(gt_bounds)

def f1(prec:float, rec:float) -> float:
    if prec and rec:
        return 2 * ((prec * rec)/(prec + rec))
    return 0



In [6]:
df['Precision'] =  df.apply(lambda x: precision(x['Bounds_predetti'], x['Bounds'], x['Repertori_predetti'], x['Repertori']), axis=1)
df['Recall'] =  df.apply(lambda x: recall(x['Bounds_predetti'], x['Bounds'], x['Repertori_predetti'], x['Repertori']), axis=1)
df['F1'] =  df.apply(lambda x: f1(x['Precision'], x['Recall']), axis=1)


In [8]:
df

,Testo,Stralci,Repertori,Bounds,Stralci_predetti,Bounds_predetti,Repertori_predetti,Precision,Recall,F1
0,"Comunque, se tutti i giornalisti che oggi si s...","[Comunque,, se tutti i giornalisti che oggi si...","[commento, valutazione, possibilità]","[(0, 0), (1, 36), (37, 41)]","[Comunque, se tutti i giornalisti che oggi si ...","[(0, 41)]",[commento],0.023810,0.333333,0.044444
1,"""Ha suscitato scandalo la proposta di Salvini ...",[Ha suscitato scandalo la proposta di Salvini ...,"[sancire, valutazione, conferma, conferma, imp...","[(0, 12), (13, 33), (34, 51), (52, 63), (64, 7...","[""Ha suscitato scandalo la proposta di Salvini...","[(0, 12), (13, 33), (34, 51), (52, 63), (64, 7...","[giudizio, opinione, giudizio, valutazione, co...",0.083333,0.083333,0.083333
2,Via via che si aggrava la crisi prodotta dalla...,[Via via che si aggrava la crisi prodotta dall...,"[causa, sancire, causa, sancire, giudizio, com...","[(0, 21), (22, 35), (36, 55), (56, 78), (79, 8...",[Via via che si aggrava la crisi prodotta dall...,"[(0, 21), (22, 55), (56, 78), (79, 129), (130,...","[causa, valutazione, contrapposizione, giudizi...",0.020648,0.080420,0.032859
3,"""In Veneto, livelli più elevati di tamponi e m...","[""In Veneto, livelli più elevati di tamponi e ...","[valutazione, sancire]","[(0, 32), (33, 53)]","[""In Veneto, livelli più elevati di tamponi e ...","[(0, 32), (33, 53)]","[valutazione, sancire]",0.038961,0.038961,0.038961
4,"Presidente, il corretto di stile di vita è tor...","[Presidente, il corretto di stile di vita è to...","[opinione, sancire]","[(0, 15), (16, 30)]","[Presidente, il corretto di stile di vita è to...","[(0, 15), (16, 30)]","[prescrizione, commento]",0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
95,Da domani oltre che guanti e mascherina cerchi...,[Da domani oltre che guanti e mascherina cerch...,"[prescrizione, implicazione]","[(0, 13), (14, 23)]",[Da domani oltre che guanti e mascherina cerch...,"[(0, 13), (14, 23)]","[prescrizione, implicazione]",0.085714,0.085714,0.085714
96,"Zaia, devi fare uno schema perché per quanto t...","[Zaia, devi fare uno schema perché per quanto ...",[commento],"[(0, 22)]","[Zaia, devi fare uno schema perché per quanto ...","[(0, 22)]",[deresponsabilizzazione],0.000000,0.000000,0.000000
97,Ricordatevi che oltre i parrucchieri e Barbier...,[Ricordatevi che oltre i parrucchieri e Barbie...,"[prescrizione, deresponsabilizzazione, giustif...","[(0, 11), (12, 19), (20, 25)]",[Ricordatevi che oltre i parrucchieri e Barbie...,"[(0, 11), (12, 15), (16, 25)]","[sancire, sancire, prescrizione]",0.000000,0.000000,0.000000
98,\nC'è troppa gente che si riunisce inventandos...,[C'è troppa gente che si riunisce inventandosi...,"[descrizione, specificazione]","[(0, 15), (16, 17)]",[\nC'è troppa gente che si riunisce inventando...,"[(0, 17)]",[sancire],0.000000,0.000000,0.000000


In [9]:
print(df['F1'].mean())

0.14057357467589496
